In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
# import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Motor neuron table and determine cell classes 
My current hierarchy would be: Region (brain,vnc,descending,ascending), segment (T1,2,3,Ab), class (motor, sensory, intersegmental, local), neurotransmitter, primary neurite bundle/hemilineage, type

In [ ]:
all_pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='clustered_ordered')
all_pre_to_mn_df['cell_class']  = ''
print(all_pre_to_mn_df.shape)

All = slice(None)
                    # segID	motor	has_sma	sensory	neck	local   T1_count
all_pre_to_mn_df.loc[(All,  True,   True,   False,  False,  False,  All),'cell_class'] = 'motor'
all_pre_to_mn_df.loc[(All,  False,  All,    True,   False,  False,  All),'cell_class'] = 'sensory' # always has those weird neurons in there
all_pre_to_mn_df.loc[(All,  False,  False,  False,  True,   False,  All),'cell_class'] = 'descending'
all_pre_to_mn_df.loc[(All,  False,  True,   False,  True,   False,  All),'cell_class'] = 'ascending'
all_pre_to_mn_df.loc[(All,  False,  False,  False,  False,  False,  All),'cell_class'] = 'fragment'
all_pre_to_mn_df.loc[(All,  False,  True,   False,  False,  True,   All),'cell_class'] = 'local'
all_pre_to_mn_df.loc[(All,  False,  True,   False,  False,  False,  All),'cell_class'] = 'intersegmental'
print(all_pre_to_mn_df.cell_class.value_counts())

all_pre_to_mn_df = all_pre_to_mn_df.set_index(keys=['cell_class'],append=True,drop=True,verify_integrity=True)
all_pre_to_mn_df = all_pre_to_mn_df.reset_index(level=['motor','has_soma','sensory','neck','local','non_T1_count'],drop=True)
all_pre_to_mn_df = all_pre_to_mn_df.reorder_levels(['cell_class','segID'],'index')
print(all_pre_to_mn_df.shape)

pre_to_mn_df = all_pre_to_mn_df.loc[(['descending','sensory','intersegmental','local','ascending']),:]
print(pre_to_mn_df.shape)

pre_to_mn_df.head()

# Note 8/2/22 - the the sensory group is currently 117. This includes 111 actual sensory neurons, likely after some proofreading. 
# Then there are 6 sensory neurons with somas. This should change on 8/3, there are still the bilateral mns, but some of the sensory snarls have been proofread

# First plot the fractional input to the MNs

In [ ]:
mn_mi = all_pre_to_mn_df.columns.to_frame()
x = utils.xticks_from_mnmi(mn_mi)

frac_df = pd.DataFrame(index=['fragment','descending','sensory','ascending','intersegmental','local','x'],columns=pre_to_mn_df.columns)
frac_df.loc['fragment',:] = all_pre_to_mn_df.loc['fragment'].sum(axis=0)
frac_df.loc['descending',:] = all_pre_to_mn_df.loc['descending'].sum(axis=0)
frac_df.loc['sensory',:] = all_pre_to_mn_df.loc['sensory'].sum(axis=0)
frac_df.loc['ascending',:] = all_pre_to_mn_df.loc['ascending'].sum(axis=0)
frac_df.loc['intersegmental',:] = all_pre_to_mn_df.loc['intersegmental'].sum(axis=0)
frac_df.loc['local',:] = all_pre_to_mn_df.loc['local'].sum(axis=0)
# frac_df.loc['motor',:] = mns.sum(axis=0)
frac_df = frac_df/frac_df.sum(axis=0)
frac_df.loc['x',:] = x

# sns.set(rc = {'figure.figsize':(15,8)})


# # locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
          #'motor', 'nont1',    'desc',     'sens',     'ascend',   'local',    'intr',     'vnc',      'fragments'
classlist = ['motor', 'descending', 'sensory', 'ascending', 'intersegmental','local',   'non_t1_motor', 'descending_post',  'sensory_post','ascending_post','neurmodulatory_ascending_neuron', 'vnc_non_premotor']
colors = ["#910951",  "#79D0F7",    "#93E5B6",  "#6084CC",  "#CC9FCC",        "#7D688E", "#CC4893",      '#79D0F7',          "#93E5B6",      "#6084CC",      "#CC4893",                          "#CC9FCC"]
classlist = ['fragment', 'descending', 'sensory', 'ascending', 'intersegmental','local',]
colors = ["#dddddd",  "#79D0F7",    "#93E5B6",  "#6084CC",  "#CC9FCC",        "#7D688E", ]
classlist = ['local', 'intersegmental', 'ascending', 'sensory', 'descending','fragment',]
colors = ["#7D688E",  "#CC9FCC",    "#6084CC",  "#93E5B6",  "#79D0F7",        "#dddddd", ]
cmap = sns.set_palette(sns.color_palette(colors))

# ax = frac_df.T.plot(x='x',kind='barh', stacked=True, legend = True,align='edge') 
ax = frac_df[::-1].T.plot(x='x',kind='bar', stacked=True, legend = False,width=1,cmap=cmap,figsize=(30,6)) 
# Most of them are pretty close to 1

# ax = frac_df.iloc[::-1].T.plot(x='x',kind='bar', stacked=True, legend = True) #, color=['aqua','red', 'steelblue','orange','yellow'])
# locs, labels = plt.xticks(ticks=[i for i in range(len(lbls))], labels=lbls, rotation=90)
plt.savefig('./figpanels/input_from_cell_classes.svg',format='svg')

In [ ]:
pre_to_mn_df.loc[:,('L','Ventral','thorax','extend')]

# Plot the total fractional input in each class 

### Pie chart  

In [ ]:
piesums = all_pre_to_mn_df.groupby(level='cell_class').sum().sum(axis=1)
piesums = piesums.loc[['local','intersegmental','motor','fragment','descending','sensory','ascending']]
print(piesums.sum())
piesums


In [ ]:
fig, ax = plt.subplots( 1, 1, figsize=(8,8))
ax.pie(piesums, explode=None, labels=piesums.index, autopct='%1.1f%%',
              shadow=False, startangle=90)

fname = './figpanels/class_input_to_mns_pie.svg'
fig.savefig(fname,format='svg')

In [ ]:
print('Total synapses: {}'.format(piesums.sum()))

# Plot the number of partners

In [ ]:
just_pre_to_mn_df = all_pre_to_mn_df.loc[(['local','intersegmental','ascending','sensory','descending'])]
just_pre_to_mn_df.astype(bool).sum(axis=0)

In [ ]:
all_pre_to_mn_df.loc[(['local','intersegmental','ascending','sensory','descending'])].sum(axis=0).max()

In [ ]:
# Double check the order of the neurons
x = utils.xticks_from_pools(just_pre_to_mn_df.columns.to_frame())
lbls = utils.mn_labels(mn_mi,depth='rank')
mat = pre_to_mn_df.to_numpy()
y = just_pre_to_mn_df.astype(bool).sum(axis=0).to_numpy()

fig = plt.figure(1, figsize=(20, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Number of descending neurons connected to each MN')
plt.xlabel('MN')
plt.ylabel('Number of inputs')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

ax1.set_ylim([0, 400])

plt.savefig('./figpanels/total_partners.svg',format='svg')

y.mean()
# y.max()

# Plot the fraction of partners in each class

In [ ]:
all_pre_to_mn_df.head()

In [ ]:
partners = all_pre_to_mn_df.loc[(['local','intersegmental','ascending','sensory','descending'])].astype(bool).groupby(by='cell_class',axis=0).sum()
partners


In [ ]:
mn_mi = partners.columns.to_frame()
x = utils.xticks_from_mnmi(mn_mi)

partners = all_pre_to_mn_df.loc[(['descending','sensory','ascending','intersegmental','local'])].astype(bool).groupby(by='cell_class',axis=0).sum()
# partners.loc['total',:] = partners.loc[['descending','sensory','ascending','intersegmental','local']].sum(axis=0)
# # partners.loc['total'] = partners.sum(axis=0)
# partners.loc['descending',:] = partners.loc['descending',:].div(partners.loc['total',:])
# partners.loc['sensory',:] = partners.loc['sensory',:].div(partners.loc['total',:])
# partners.loc['ascending',:] = partners.loc['ascending',:].div(partners.loc['total',:])
# partners.loc['intersegmental',:] = partners.loc['intersegmental',:].div(partners.loc['total',:])
# partners.loc['local',:] = partners.loc['local',:].div(partners.loc['total',:])

partners = partners.div(partners.sum(axis=0))
partners = partners.loc[['descending','sensory','ascending','intersegmental','local'],:]
partners

classlist = ['local', 'intersegmental', 'ascending', 'sensory', 'descending','fragment',]
colors = ["#7D688E",  "#CC9FCC",    "#6084CC",  "#93E5B6",  "#79D0F7",        "#dddddd", ]
cmap = sns.set_palette(sns.color_palette(colors))

# ax = frac_df.T.plot(x='x',kind='barh', stacked=True, legend = True,align='edge') 
ax = partners[::-1].T.plot(kind='bar', stacked=True, legend = False,width=1,cmap=cmap,figsize=(30,6)) 
# Most of them are pretty close to 1

# ax = frac_df.iloc[::-1].T.plot(x='x',kind='bar', stacked=True, legend = True) #, color=['aqua','red', 'steelblue','orange','yellow'])
# locs, labels = plt.xticks(ticks=[i for i in range(len(lbls))], labels=lbls, rotation=90)
plt.savefig('./figpanels/prepartners_from_cell_classes.svg',format='svg')

In [ ]:

frac_df = pd.DataFrame(index=['descending','sensory','ascending','intersegmental','local','x'],columns=partners.columns)
frac_df.loc['descending',:] = all_pre_to_mn_df.loc['descending'].sum(axis=0)
frac_df.loc['sensory',:] = all_pre_to_mn_df.loc['sensory'].sum(axis=0)
frac_df.loc['ascending',:] = all_pre_to_mn_df.loc['ascending'].sum(axis=0)
frac_df.loc['intersegmental',:] = all_pre_to_mn_df.loc['intersegmental'].sum(axis=0)
frac_df.loc['local',:] = all_pre_to_mn_df.loc['local'].sum(axis=0)
# frac_df.loc['motor',:] = mns.sum(axis=0)
frac_df = frac_df/frac_df.sum(axis=0)
frac_df.loc['x',:] = x

# sns.set(rc = {'figure.figsize':(15,8)})


# # locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
          #'motor', 'nont1',    'desc',     'sens',     'ascend',   'local',    'intr',     'vnc',      'fragments'
classlist = ['motor', 'descending', 'sensory', 'ascending', 'intersegmental','local',   'non_t1_motor', 'descending_post',  'sensory_post','ascending_post','neurmodulatory_ascending_neuron', 'vnc_non_premotor']
colors = ["#910951",  "#79D0F7",    "#93E5B6",  "#6084CC",  "#CC9FCC",        "#7D688E", "#CC4893",      '#79D0F7',          "#93E5B6",      "#6084CC",      "#CC4893",                          "#CC9FCC"]
classlist = ['fragment', 'descending', 'sensory', 'ascending', 'intersegmental','local',]
colors = ["#dddddd",  "#79D0F7",    "#93E5B6",  "#6084CC",  "#CC9FCC",        "#7D688E", ]
classlist = ['local', 'intersegmental', 'ascending', 'sensory', 'descending','fragment',]
colors = ["#7D688E",  "#CC9FCC",    "#6084CC",  "#93E5B6",  "#79D0F7",        "#dddddd", ]
cmap = sns.set_palette(sns.color_palette(colors))

# ax = frac_df.T.plot(x='x',kind='barh', stacked=True, legend = True,align='edge') 
ax = frac_df[::-1].T.plot(x='x',kind='bar', stacked=True, legend = False,width=1,cmap=cmap,figsize=(30,6)) 
# Most of them are pretty close to 1

# ax = frac_df.iloc[::-1].T.plot(x='x',kind='bar', stacked=True, legend = True) #, color=['aqua','red', 'steelblue','orange','yellow'])
# locs, labels = plt.xticks(ticks=[i for i in range(len(lbls))], labels=lbls, rotation=90)
plt.savefig('./figpanels/input_from_cell_classes.svg',format='svg')

# Purpose of this script
This script answers the following question: Is there a stable match where we can assign a local premotor neuron to a muscle group?
The idea comes from the med school matching algorithm and aims to produce an order to the premotor neurons where the "most important" neurons for a given muscle is assigned to that muscle. This will allow us to look at the connectivity of the local premotors and say, e.g., "this important neuron for tibia flexors is also connected to femur flexors". So, what does "important" mean? To assign premotors to motor pools, for each premotor neuron, we rank the motor pools by the number of combined inputs to the motor pool. For each motor pool, we rank premotor neurons by their contribution to the cluster unit vector. Then we let the matching algorithm work on those rankings. This ranking system is premotor/student optimal, and the lists for the motor pool/school are not limited, so it should aggregate premotor neurons just to their prefered pool.

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()

mn_mi = all_pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')


fig = plt.figure(1, figsize = [15,25])
ax = sns.heatmap(np.log10(all_pre_to_mn_df.to_numpy()+1), xticklabels=lbls, cmap=cmap)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neuron SegIDS', fontsize =18)
plt.ylabel('Hair Plate Identity', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

#fig.savefig('Ordered_unsorted.svg',format='svg')

# Match premns to motor pools

In [ ]:
from match.RothPeranson import MatchController
plot_logical = False
show_logical = False

new_pre_to_mn = None
sho_pre_to_mn = None

pool_capacity = 65  # This limits how many neurons make up the main tibia flexor pool
pool_capacity = 200 # This allows nearly as many premotor neurons into any pool that want to

mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',
    # 'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]

# quick sanity check, all the mns assigned to a pool
# total=0
# for key in pool_keys:
#     a = pre_to_mn_df.loc[:,mpool_dict[key]].shape[1]
#     total = total+a
#     print('{}: {}'.format(key,a))
# print('{} total'.format(total))

for cclass_tup in [('descending',All,), ('sensory',All,), ('ascending',All,),('intersegmental',All,),('local',All,)]:
# for cclass_tup in [('local',All,)]:

    cell_class = pre_to_mn_df.loc[cclass_tup,:]

    figfname = './figpanels/{}_match_matrix.svg'.format(cclass_tup[0])
    ylabelstring = '{} neurons'.format(cclass_tup[0])
    print('Starting {}, {} premotor neurons'.format(ylabelstring,cell_class.shape[0]))

    # Make a data frame of inputs to a pool
    pre_to_pool_df = None
    for key in pool_keys:
        if pre_to_pool_df is None:
            pre_to_pool_df = cell_class.loc[:,mpool_dict[key]].sum(axis=1)
        else:    
            pre_to_pool_df = pd.concat([pre_to_pool_df,cell_class.loc[:,mpool_dict[key]].sum(axis=1)],axis=1)
    pre_to_pool_df.columns = pool_keys
    pre_to_pool_df

    # Rank the motor pools for each premotor neuron
    pre_to_pool_ranks_df = pre_to_pool_df.assign(**pre_to_pool_df.iloc[:, 0:].rank(axis = 1, ascending = False).astype(int))
    pre_to_pool_ranks_df.index = pre_to_pool_ranks_df.index.get_level_values('segID')

    # take the transpose, so each premotor neuron is a column and fill in the values with the ranked pool
    premn_prefs_for_pools = pre_to_pool_ranks_df.T
    pools = np.array(premn_prefs_for_pools.index.to_list())
    for idx in premn_prefs_for_pools:
            premn_prefs_for_pools[idx] = pools[(premn_prefs_for_pools[idx].to_numpy(dtype=int)- 1).argsort()]
    premn_prefs_for_pools = premn_prefs_for_pools.reset_index(drop=True)
    premn_prefs_for_pools.columns = premn_prefs_for_pools.columns.astype(str)

    # Now create a DF where the motor pools rank the premotor neurons. 
    # Do this by ranking along the pool

    pool_ranks_of_pre_df = pre_to_pool_df.assign(**pre_to_pool_df.iloc[0:,:].rank(axis = 0, ascending = False).astype(int))
    pool_ranks_of_pre_df.index = pool_ranks_of_pre_df.index.get_level_values('segID')

    # rank the premotors
    pool_prefs_for_premns = pool_ranks_of_pre_df
    premns = np.array(pool_prefs_for_premns.index.astype(str).to_list())
    for idx in pool_prefs_for_premns:
            pool_prefs_for_premns[idx] = premns[(pool_prefs_for_premns[idx].to_numpy(dtype=int)- 1).argsort()]
    pool_prefs_for_premns = pool_prefs_for_premns.reset_index(drop=True)

    pool_capacities = pool_prefs_for_premns.count()-(pool_prefs_for_premns.shape[0]-pool_capacity)
    pool_capacities_df = pool_capacities.to_frame()

    pool_capacities_df = pool_capacities_df.rename(columns = {0: 'places'})

    ## Now run the match algorithm
    match = MatchController(pool_prefs_for_premns, premn_prefs_for_pools,pool_capacities_df)
    match.start_match()
    match_dict = match.results_dict()

    # Now reorder according to the match
    lidx = cell_class.index.to_frame()
    lidx['preferred_pool'] = ''
    for idx,row in lidx.iterrows():
        lidx.loc[idx,'preferred_pool'] = match_dict[str(row.segID)]
    lidx
        
    cell_class.index = pd.MultiIndex.from_frame(lidx)
    cell_class = cell_class.reorder_levels(['cell_class','preferred_pool','segID'],'index')

    s = cell_class.sum(axis=1)
    cell_class = cell_class.loc[s.sort_values(ascending=False).index,:]
    cell_class = cell_class.loc[(All,pool_keys,All),:]

    cell_class.head()

    %config InlineBackend.figure_formats = ['png']
    %matplotlib inline

    if plot_logical:
        cmap = utils.white_dense()

        mn_mi = cell_class.columns.to_frame()
        lbls = utils.mn_labels(mn_mi,depth='rank')
        fig = plt.figure(1, figsize = [15,25])
        ax = sns.heatmap(np.log10(cell_class.to_numpy()+1), xticklabels=lbls, cmap=cmap)
        cbar = ax.collections[0].colorbar
        # here set the labelsize by 20
        # cbar.ax.tick_params(labelsize=20)
        cbar.set_label(label = 'log 10 # of synapses', size=24)
        plt.xlabel('Motor Neurons', fontsize =18)
        plt.ylabel(ylabelstring, fontsize =18)
        plt.yticks(fontsize = 16)
        plt.xticks(fontsize = 16)
        plt.show()

        fig.savefig(figfname,format='svg')

    if show_logical:
        # put in a gap, just for show
        cell_class_for_show = None
        pool_gap = cell_class.iloc[:,0].copy().to_frame()
        for col in pool_gap.columns:
            pool_gap[col].values[:] = 2

        cell_class_for_show = None
        for key in pool_keys:
            if cell_class_for_show is None:
                cell_class_for_show = cell_class.loc[:,mpool_dict[key]].copy()
            else:    
                cell_class_for_show = pd.concat([cell_class_for_show,pool_gap.copy()],axis=1)
                cell_class_for_show = pd.concat([cell_class_for_show,cell_class.loc[:,mpool_dict[key]]],axis=1)
    
        class_gap = cell_class_for_show.iloc[0:10,:].copy()
        for col in class_gap.columns:
            class_gap[col].values[:] = 2
        if sho_pre_to_mn is None:
            sho_pre_to_mn = cell_class_for_show.copy()
        else:
            sho_pre_to_mn = pd.concat([sho_pre_to_mn,class_gap.copy()],axis=0)
            sho_pre_to_mn = pd.concat([sho_pre_to_mn,cell_class_for_show],axis=0)

    if new_pre_to_mn is None:
        new_pre_to_mn = cell_class
    else:
        new_pre_to_mn = pd.concat([new_pre_to_mn,cell_class],axis=0)


if show_logical:
    fig = plt.figure(1, figsize = [15,25])
    ax = sns.heatmap(np.log10(sho_pre_to_mn.to_numpy()+1), xticklabels=lbls, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Motor Neurons', fontsize =18)
    plt.ylabel(ylabelstring, fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()

    fig.savefig('matched_premotor_to_motor',format='svg')

In [ ]:
pool_keys

In [ ]:
# lidx.loc[(All,648518346475845090)]
# lidx
# lidx2 = pd.MultiIndex.from_frame(lidx)
# lidx3 = lidx2.to_frame()
# # lidx3.loc[(All,648518346475845090)]
# cell_class.index = pd.MultiIndex.from_frame(lidx)
# cell_class.loc[(All,648518346475845090,All)]
# cell_class.shape
# (lidx.segID==648518346475845090).sum()

# lidx.set_index('segID',drop=True).loc[(648518346475845090)]
# lidx.loc[lidx.segID==648518346475845090]
# lidx.loc[('local',648518346475845090)]

In [ ]:
fig = plt.figure(1, figsize = [8,20])
import cmocean
cmap_alt = cmocean.cm.dense
ax = sns.heatmap(np.log10(new_pre_to_mn.to_numpy()+1), xticklabels=lbls, cmap=cmap_alt)
ax.set_rasterized(True)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel(ylabelstring, fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
# plt.show()

plt.xlabel(None)
plt.ylabel(None)
plt.xticks([])
plt.yticks([])
plt.show()


fig.savefig('./figpanels/matched_premotor_to_motor.eps',format='eps',dpi=300, bbox_inches='tight')
# plt.savefig('wingConnMatrix0110.eps', format='eps', bbox_inches='tight')
# fig.savefig('./figpanels/matched_premotor_to_motor.png',format='png')

In [ ]:
connectome_create.save_pre_to_mn_df(new_pre_to_mn,ext='matched_to_pool')